In [ ]:
import random
import pandas as pd
from datetime import datetime
from playwright.async_api import async_playwright
import asyncio

async def scrape_with_retries(leagues, start_year=2004, end_year=2025, max_retries=20):
    df_results = []
    remaining = []

    for league_name, base_url in leagues.items():
        for year in range(start_year, end_year + 1):
            full_url = f"{base_url}?saison_id={year}"
            remaining.append({"league": league_name, "year": year, "url": full_url})

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
            locale="de-DE"
        )
        page = await context.new_page()

        attempt = 1
        while remaining and attempt <= max_retries:
            print(f"\n🔁 Attempt {attempt} with {len(remaining)} URLs to process...")
            current_batch = remaining
            remaining = []

            for entry in current_batch:
                league = entry["league"]
                year = entry["year"]
                url = entry["url"]

                try:
                    print(f"🌐 {league} {year}: Loading {url}")
                    await page.goto(url, timeout=60000)
                    await page.wait_for_load_state("networkidle", timeout=30000)
                    await page.wait_for_timeout(random.randint(1000, 3000))

                    content = await page.content()
                    if "Keine Daten verfügbar" in content:
                        print(f"🚫 {league} {year}: No data available.")
                        continue

                    xpath = '/html/body/div[2]/main/div[1]/div[1]/div[2]/div[2]/div/table/tfoot/tr/td[6]'
                    locator = page.locator(f'xpath={xpath}')
                    await locator.wait_for(timeout=5000)

                    value = await locator.text_content()
                    if value:
                        value_clean = value.strip()
                        print(f"✅ {league} {year}: {value_clean}")
                        df_results.append({
                            "league": league,
                            "year": year,
                            "value": value_clean
                        })
                    else:
                        print(f"⚠️ {league} {year}: XPath found, but empty.")
                        remaining.append(entry)

                except Exception as e:
                    print(f"❌ {league} {year}: Failed — {str(e)}")
                    remaining.append(entry)

            attempt += 1
        await browser.close()

    return pd.DataFrame(df_results)

async def main():
    leagues = {
        "MLS Soccer ": "https://www.transfermarkt.de/major-league-soccer/startseite/wettbewerb/MLS1/plus/",
        "Ligue 1": "https://www.transfermarkt.de/ligue-1/startseite/wettbewerb/FR1/plus/",
        "Saudi Pro League": "https://www.transfermarkt.de/saudi-pro-league/startseite/wettbewerb/SA1/plus/",
        "Bundesliga": "https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/",
        "2. Bundesliga": "https://www.transfermarkt.de/2-bundesliga/startseite/wettbewerb/L2/plus/",
        "Premier League": "https://www.transfermarkt.de/premier-league/startseite/wettbewerb/GB1/plus/",
        "Championship": "https://www.transfermarkt.de/championship/startseite/wettbewerb/GB2/plus/",
        "La Liga": "https://www.transfermarkt.de/laliga/startseite/wettbewerb/ES1/plus/",
        "Serie A": "https://www.transfermarkt.de/serie-a/startseite/wettbewerb/IT1/plus/"
           }
    print("🌍 Starting transfermarkt scraper...")

    df = await scrape_with_retries(leagues, start_year=2004, end_year=2025)

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    df.to_excel(f"transfermarkt_results_{timestamp}.xlsx", index=False)
    print("📁 Data saved.")

await main()